# Taller 2 - Procesamiento de Señales Intracelulares y Extracelulares

Este taller involucra el procesamiento de señales electrofisiológicas usando detección de picos con `findpeaks` y filtrado de señales.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from scipy.io import loadmat
from scipy.signal import find_peaks, butter, filtfilt, freqz
import ipywidgets as widgets
from IPython.display import display

# Configurar plotly para notebook
pio.renderers.default = "notebook"

## 1. Cargar datos del archivo .mat

In [ ]:
# Cargar el archivo .mat
data = loadmat('IntraExtracelular_py.mat')

# Explorar las variables
print("Claves en el archivo .mat:")
for key in data.keys():
    if not key.startswith('__'):
        print(f"{key}: {type(data[key])}, shape: {data[key].shape if hasattr(data[key], 'shape') else 'N/A'}")

# Extraer las variables principales
variable1 = data['variable1']  # Matriz de 2 canales (intracelular y extracelular)
variable2 = data['variable2']  # Estructura con información del archivo

print(f"\nvariable1 shape: {variable1.shape}")
print(f"variable2 type: {type(variable2)}")

# Extraer frecuencia de muestreo
fs = float(variable2[0][0][0][0])  # Navegar la estructura anidada
print(f"Frecuencia de muestreo: {fs} Hz")

# Separar las señales
señal_intracelular = variable1[:, 0]  # Primer canal
señal_extracelular = variable1[:, 1]  # Segundo canal

# Crear vector de tiempo
n_muestras = len(señal_intracelular)
tiempo = np.arange(n_muestras) / fs

print(f"\nNúmero de muestras: {n_muestras}")
print(f"Duración de la señal: {tiempo[-1]:.2f} segundos")
print(f"Rango señal intracelular: [{señal_intracelular.min():.3f}, {señal_intracelular.max():.3f}]")
print(f"Rango señal extracelular: [{señal_extracelular.min():.3f}, {señal_extracelular.max():.3f}]")

## 2. Graficar señales intracelular y extracelular

In [ ]:
# Crear subplots interactivos con plotly
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Señal Intracelular', 'Señal Extracelular'),
    shared_xaxes=True,
    vertical_spacing=0.1
)

# Añadir señal intracelular
fig.add_trace(
    go.Scatter(
        x=tiempo,
        y=señal_intracelular,
        name='Intracelular',
        line=dict(color='blue', width=0.5)
    ),
    row=1, col=1
)

# Añadir señal extracelular
fig.add_trace(
    go.Scatter(
        x=tiempo,
        y=señal_extracelular,
        name='Extracelular',
        line=dict(color='green', width=0.5)
    ),
    row=2, col=1
)

# Configurar layout
fig.update_layout(
    title='Señales Electrofisiológicas',
    xaxis2_title='Tiempo (s)',
    yaxis_title='Amplitud (V)',
    yaxis2_title='Amplitud (V)',
    height=700,
    showlegend=True
)

# Hacer que los ejes x estén sincronizados (linkaxes equivalente)
fig.update_xaxes(matches='x')

fig.show()

## 3. Detección de picos en señal intracelular

In [ ]:
# Calcular umbral estadístico para la señal intracelular
media_intra = np.mean(señal_intracelular)
std_intra = np.std(señal_intracelular)
umbral_intra = media_intra + 3 * std_intra  # Umbral a 3 desviaciones estándar

print(f"Estadísticas señal intracelular:")
print(f"Media: {media_intra:.6f} V")
print(f"Desviación estándar: {std_intra:.6f} V")
print(f"Umbral (media + 3*std): {umbral_intra:.6f} V")

# Detectar picos usando find_peaks con prominence
# El parámetro 'height' define el umbral mínimo
# El parámetro 'prominence' ayuda a filtrar picos más significativos
picos_intra, propiedades_intra = find_peaks(
    señal_intracelular, 
    height=umbral_intra,
    prominence=std_intra,  # Usar prominence para mejor detección
    distance=int(0.001 * fs)  # Distancia mínima entre picos (1ms)
)

print(f"\nPicos detectados en señal intracelular: {len(picos_intra)}")
print(f"Alturas de los picos: rango [{propiedades_intra['peak_heights'].min():.6f}, {propiedades_intra['peak_heights'].max():.6f}] V")
print(f"Prominencias: rango [{propiedades_intra['prominences'].min():.6f}, {propiedades_intra['prominences'].max():.6f}] V")

In [ ]:
# Crear vector binario de picos
vector_picos_intra = np.zeros_like(señal_intracelular)
vector_picos_intra[picos_intra] = 1

print(f"Vector de picos creado: {np.sum(vector_picos_intra)} picos marcados")

# Visualización interactiva de la señal intracelular con picos
fig_intra = go.Figure()

# Señal original
fig_intra.add_trace(
    go.Scatter(
        x=tiempo,
        y=señal_intracelular,
        name='Señal Intracelular',
        line=dict(color='blue', width=1)
    )
)

# Línea de umbral
fig_intra.add_hline(
    y=umbral_intra, 
    line_dash="dash", 
    line_color="orange",
    annotation_text=f"Umbral: {umbral_intra:.6f} V"
)

# Picos detectados
fig_intra.add_trace(
    go.Scatter(
        x=tiempo[picos_intra],
        y=señal_intracelular[picos_intra],
        mode='markers',
        name=f'Picos ({len(picos_intra)})',
        marker=dict(color='red', size=6, symbol='circle')
    )
)

fig_intra.update_layout(
    title='Detección de Picos - Señal Intracelular',
    xaxis_title='Tiempo (s)',
    yaxis_title='Amplitud (V)',
    height=500
)

fig_intra.show()

## 4. Procesamiento de señal extracelular

In [ ]:
# Diseño de filtro para señal extracelular
# Los spikes extracelulares suelen estar en frecuencias altas (300-3000 Hz)

# Parámetros del filtro pasa-altas
fc_low = 300  # Frecuencia de corte baja (Hz)
fc_high = 3000  # Frecuencia de corte alta (Hz)
nyquist = fs / 2

# Crear filtro pasa-banda Butterworth de 4to orden
orden = 4
low = fc_low / nyquist
high = fc_high / nyquist

b, a = butter(orden, [low, high], btype='band')

print(f"Filtro pasa-banda diseñado:")
print(f"Frecuencia de corte baja: {fc_low} Hz")
print(f"Frecuencia de corte alta: {fc_high} Hz")
print(f"Orden del filtro: {orden}")
print(f"Frecuencia de Nyquist: {nyquist} Hz")

# Ver respuesta en frecuencia del filtro
w, h = freqz(b, a, worN=8000)
freq_hz = w * fs / (2 * np.pi)

fig_filter = go.Figure()
fig_filter.add_trace(
    go.Scatter(
        x=freq_hz,
        y=20 * np.log10(abs(h)),
        name='Respuesta en frecuencia',
        line=dict(color='purple')
    )
)

fig_filter.add_vline(x=fc_low, line_dash="dash", line_color="red", annotation_text=f"fc_low = {fc_low} Hz")
fig_filter.add_vline(x=fc_high, line_dash="dash", line_color="red", annotation_text=f"fc_high = {fc_high} Hz")

fig_filter.update_layout(
    title='Respuesta en Frecuencia del Filtro Pasa-Banda',
    xaxis_title='Frecuencia (Hz)',
    yaxis_title='Magnitud (dB)',
    xaxis=dict(range=[0, 5000])
)

fig_filter.show()

In [ ]:
# Aplicar filtro usando filtfilt (corrección automática de fase)
señal_extra_filtrada = filtfilt(b, a, señal_extracelular)

print(f"Señal extracelular filtrada:")
print(f"Rango original: [{señal_extracelular.min():.6f}, {señal_extracelular.max():.6f}] V")
print(f"Rango filtrada: [{señal_extra_filtrada.min():.6f}, {señal_extra_filtrada.max():.6f}] V")

# Comparar señal original vs filtrada
fig_comp = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Señal Extracelular Original', 'Señal Extracelular Filtrada'),
    shared_xaxes=True
)

# Usar un subconjunto de datos para mejor visualización
t_inicio, t_fin = 10, 15  # Mostrar 5 segundos
idx_inicio = int(t_inicio * fs)
idx_fin = int(t_fin * fs)

fig_comp.add_trace(
    go.Scatter(
        x=tiempo[idx_inicio:idx_fin],
        y=señal_extracelular[idx_inicio:idx_fin],
        name='Original',
        line=dict(color='green', width=1)
    ),
    row=1, col=1
)

fig_comp.add_trace(
    go.Scatter(
        x=tiempo[idx_inicio:idx_fin],
        y=señal_extra_filtrada[idx_inicio:idx_fin],
        name='Filtrada',
        line=dict(color='red', width=1)
    ),
    row=2, col=1
)

fig_comp.update_layout(
    title='Comparación: Señal Extracelular Original vs Filtrada',
    xaxis2_title='Tiempo (s)',
    yaxis_title='Amplitud (V)',
    yaxis2_title='Amplitud (V)',
    height=600
)

fig_comp.show()

## 5. Detección de picos en señal extracelular (picos negativos)

In [ ]:
# Para detectar picos negativos, invertimos la señal
# Los spikes extracelulares suelen ser deflexiones negativas
señal_extra_invertida = -señal_extra_filtrada

# Calcular umbral estadístico para la señal extracelular filtrada
media_extra = np.mean(señal_extra_filtrada)
std_extra = np.std(señal_extra_filtrada)

# Para picos negativos, el umbral debe ser negativo
umbral_extra = media_extra - 4 * std_extra  # Umbral más estricto para extracelular

print(f"Estadísticas señal extracelular filtrada:")
print(f"Media: {media_extra:.6f} V")
print(f"Desviación estándar: {std_extra:.6f} V")
print(f"Umbral para picos negativos (media - 4*std): {umbral_extra:.6f} V")

# Detectar picos en la señal invertida (para encontrar picos negativos)
picos_extra, propiedades_extra = find_peaks(
    señal_extra_invertida,
    height=-umbral_extra,  # Invertir el umbral
    prominence=2*std_extra,  # Prominence más alta para extracelular
    distance=int(0.001 * fs)  # Distancia mínima de 1ms
)

print(f"\nPicos detectados en señal extracelular: {len(picos_extra)}")
if len(picos_extra) > 0:
    alturas_reales = señal_extra_filtrada[picos_extra]  # Alturas en la señal original (negativas)
    print(f"Alturas de los picos (valores negativos): rango [{alturas_reales.min():.6f}, {alturas_reales.max():.6f}] V")
    print(f"Prominencias: rango [{propiedades_extra['prominences'].min():.6f}, {propiedades_extra['prominences'].max():.6f}] V")

# Crear vector binario de picos extracelulares
vector_picos_extra = np.zeros_like(señal_extracelular)
vector_picos_extra[picos_extra] = -1  # Usar -1 para indicar picos negativos

print(f"Vector de picos extracelulares creado: {np.sum(np.abs(vector_picos_extra))} picos marcados")

In [ ]:
# Visualización de la detección de picos en señal extracelular
fig_extra = go.Figure()

# Mostrar un segmento más pequeño para mejor visualización
t_start, t_end = 20, 25  # 5 segundos
idx_start = int(t_start * fs)
idx_end = int(t_end * fs)

# Señal filtrada
fig_extra.add_trace(
    go.Scatter(
        x=tiempo[idx_start:idx_end],
        y=señal_extra_filtrada[idx_start:idx_end],
        name='Señal Extracelular Filtrada',
        line=dict(color='green', width=1)
    )
)

# Línea de umbral
fig_extra.add_hline(
    y=umbral_extra,
    line_dash="dash",
    line_color="orange",
    annotation_text=f"Umbral: {umbral_extra:.6f} V"
)

# Filtrar picos que están en el rango de visualización
picos_en_rango = picos_extra[(picos_extra >= idx_start) & (picos_extra < idx_end)]

if len(picos_en_rango) > 0:
    # Picos detectados
    fig_extra.add_trace(
        go.Scatter(
            x=tiempo[picos_en_rango],
            y=señal_extra_filtrada[picos_en_rango],
            mode='markers',
            name=f'Picos Negativos ({len(picos_en_rango)} en rango)',
            marker=dict(color='red', size=8, symbol='circle')
        )
    )

fig_extra.update_layout(
    title=f'Detección de Picos Negativos - Señal Extracelular (t={t_start}-{t_end}s)',
    xaxis_title='Tiempo (s)',
    yaxis_title='Amplitud (V)',
    height=500
)

fig_extra.show()

## 6. Visualización final interactiva

In [ ]:
# Crear visualización final con todas las señales y detecciones
fig_final = make_subplots(
    rows=4, cols=1,
    subplot_titles=(
        'Señal Intracelular con Picos Detectados',
        'Señal Extracelular Original',
        'Señal Extracelular Filtrada con Picos Detectados',
        'Señales Binarias de Picos'
    ),
    shared_xaxes=True,
    vertical_spacing=0.05
)

# Subconjunto de datos para visualización (primeros 30 segundos)
t_max = 30
idx_max = int(t_max * fs)
t_vis = tiempo[:idx_max]
intra_vis = señal_intracelular[:idx_max]
extra_vis = señal_extracelular[:idx_max]
extra_filt_vis = señal_extra_filtrada[:idx_max]
picos_intra_vis = vector_picos_intra[:idx_max]
picos_extra_vis = vector_picos_extra[:idx_max]

# Filtrar picos en el rango de visualización
picos_intra_rango = picos_intra[picos_intra < idx_max]
picos_extra_rango = picos_extra[picos_extra < idx_max]

# 1. Señal intracelular con picos
fig_final.add_trace(
    go.Scatter(
        x=t_vis, y=intra_vis,
        name='Intracelular',
        line=dict(color='blue', width=0.8)
    ), row=1, col=1
)

if len(picos_intra_rango) > 0:
    fig_final.add_trace(
        go.Scatter(
            x=tiempo[picos_intra_rango],
            y=señal_intracelular[picos_intra_rango],
            mode='markers',
            name=f'Picos Intra ({len(picos_intra_rango)})',
            marker=dict(color='red', size=4)
        ), row=1, col=1
    )

# 2. Señal extracelular original
fig_final.add_trace(
    go.Scatter(
        x=t_vis, y=extra_vis,
        name='Extra Original',
        line=dict(color='green', width=0.8)
    ), row=2, col=1
)

# 3. Señal extracelular filtrada con picos
fig_final.add_trace(
    go.Scatter(
        x=t_vis, y=extra_filt_vis,
        name='Extra Filtrada',
        line=dict(color='darkgreen', width=0.8)
    ), row=3, col=1
)

if len(picos_extra_rango) > 0:
    fig_final.add_trace(
        go.Scatter(
            x=tiempo[picos_extra_rango],
            y=señal_extra_filtrada[picos_extra_rango],
            mode='markers',
            name=f'Picos Extra ({len(picos_extra_rango)})',
            marker=dict(color='red', size=4)
        ), row=3, col=1
    )

# 4. Señales binarias
fig_final.add_trace(
    go.Scatter(
        x=t_vis, y=picos_intra_vis,
        name='Picos Intra Binarios',
        line=dict(color='red', width=1)
    ), row=4, col=1
)

fig_final.add_trace(
    go.Scatter(
        x=t_vis, y=picos_extra_vis,
        name='Picos Extra Binarios',
        line=dict(color='purple', width=1)
    ), row=4, col=1
)

# Configuración final
fig_final.update_layout(
    title='Análisis Completo de Señales Electrofisiológicas',
    height=1000,
    showlegend=True
)

fig_final.update_xaxes(title_text="Tiempo (s)", row=4, col=1)
fig_final.update_yaxes(title_text="Amplitud (V)", row=1, col=1)
fig_final.update_yaxes(title_text="Amplitud (V)", row=2, col=1)
fig_final.update_yaxes(title_text="Amplitud (V)", row=3, col=1)
fig_final.update_yaxes(title_text="Binario", row=4, col=1)

fig_final.show()

## 7. Resumen de resultados y análisis

In [ ]:
print("=" * 60)
print("RESUMEN DEL ANÁLISIS DE SEÑALES ELECTROFISIOLÓGICAS")
print("=" * 60)

print(f"\n📊 DATOS GENERALES:")
print(f"   • Frecuencia de muestreo: {fs:.0f} Hz")
print(f"   • Número de muestras: {n_muestras:,}")
print(f"   • Duración total: {tiempo[-1]:.2f} segundos")

print(f"\n🔵 SEÑAL INTRACELULAR:")
print(f"   • Rango de amplitud: [{señal_intracelular.min():.6f}, {señal_intracelular.max():.6f}] V")
print(f"   • Media: {media_intra:.6f} V")
print(f"   • Desviación estándar: {std_intra:.6f} V")
print(f"   • Umbral utilizado: {umbral_intra:.6f} V")
print(f"   • Picos detectados: {len(picos_intra)}")
print(f"   • Frecuencia de picos: {len(picos_intra)/(tiempo[-1]/60):.1f} picos/minuto")

print(f"\n🟢 SEÑAL EXTRACELULAR:")
print(f"   • Rango original: [{señal_extracelular.min():.6f}, {señal_extracelular.max():.6f}] V")
print(f"   • Rango filtrada: [{señal_extra_filtrada.min():.6f}, {señal_extra_filtrada.max():.6f}] V")
print(f"   • Filtro aplicado: Pasa-banda {fc_low}-{fc_high} Hz, orden {orden}")
print(f"   • Media (filtrada): {media_extra:.6f} V")
print(f"   • Desviación estándar (filtrada): {std_extra:.6f} V")
print(f"   • Umbral utilizado: {umbral_extra:.6f} V")
print(f"   • Picos negativos detectados: {len(picos_extra)}")
print(f"   • Frecuencia de picos: {len(picos_extra)/(tiempo[-1]/60):.1f} picos/minuto")

print(f"\n🔧 PARÁMETROS DE DETECCIÓN:")
print(f"   • Prominence intracelular: {std_intra:.6f} V")
print(f"   • Prominence extracelular: {2*std_extra:.6f} V")
print(f"   • Distancia mínima entre picos: {int(0.001 * fs)} muestras (1 ms)")

print(f"\n✅ ANÁLISIS COMPLETADO")
print(f"   • find_peaks utilizado con parámetros height y prominence")
print(f"   • Filtrado con filtfilt para corrección automática de fase")
print(f"   • Estrategia para picos negativos: inversión de señal")
print(f"   • Visualizaciones interactivas generadas con Plotly")

if len(picos_intra) > 0 and len(picos_extra) > 0:
    ratio = len(picos_extra) / len(picos_intra)
    print(f"\n📈 COMPARACIÓN:")
    print(f"   • Ratio picos extra/intra: {ratio:.2f}")
    if ratio > 2:
        print(f"   • La señal extracelular muestra mayor actividad de picos")
    elif ratio < 0.5:
        print(f"   • La señal intracelular muestra mayor actividad de picos")
    else:
        print(f"   • Actividad de picos similar entre ambas señales")